In [ ]:
#Import libraries
import pandas as pd
from bs4 import BeautifulSoup as bs
import requests

In [ ]:
# Inserting the amazon link
amazon='https://www.amazon.in/'
reviews='https://www.amazon.in/HP-Laptop-14-inch-Windows-14s-dy2500TU/product-reviews/B08XY3QXY5/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews'

In [ ]:
customer_name = []
review_title = []
rate = []
review_content = []

In [ ]:
tt = 0
while tt == 0:
    page = requests.get(reviews)
    while page.ok == False:  
        page = requests.get(reviews)
    
    
    soup = bs(page.content,'html.parser')
    soup.prettify()
    
    names = soup.find_all('span', class_='a-profile-name')
    names.pop(0)
    names.pop(0)
    
    for i in range(0,len(names)):
        customer_name.append(names[i].get_text())
        
    title = soup.find_all("a",{"data-hook":"review-title"})
    for i in range(0,len(title)):
        review_title.append(title[i].get_text())

    rating = soup.find_all('i',class_='review-rating')
    rating.pop(0)
    rating.pop(0)
    for i in range(0,len(rating)):
        rate.append(rating[i].get_text())

    review = soup.find_all("span",{"data-hook":"review-body"})
    for i in range(0,len(review)):
        review_content.append(review[i].get_text())
        
    try:
        for div in soup.findAll('li', attrs={'class':'a-last'}):
            A = div.find('a')['href']
        reviews = amazon + A
    except:
        break

In [ ]:
print(soup.prettify())

In [ ]:
len(customer_name)

In [ ]:
len(review_title)

In [ ]:
len(rate)

In [ ]:
len(review_content)

In [ ]:
review_title[:] = [titles.lstrip('\n') for titles in review_title]

review_title[:] = [titles.rstrip('\n') for titles in review_title]

review_content[:] = [titles.lstrip('\n') for titles in review_content]

review_content[:] = [titles.rstrip('\n') for titles in review_content]

In [ ]:
amazon = pd.DataFrame()

In [ ]:
amazon['Customer Name'] = customer_name
amazon['Review Title'] = review_title
amazon['Rating'] = rate
amazon['Reviews'] = review_content

In [ ]:
amazon.head()

In [ ]:
amazon.to_csv('amazon.csv',index = True)

In [ ]:
data = pd.read_csv("amazon.csv",index_col=[0])

In [ ]:
data.head()

In [ ]:
data['text']=data['Review Title']+' '+data['Reviews']

In [ ]:
data

In [ ]:
import re

In [ ]:
re_without_space = re.compile("(\.)|(\,)|(\{)|(\})|(\()|(\))|(\[)|(\])|(\;)|(\:)|(\!)|(\`)|(\')|(\")|(\%)|(\$)|(\<)|(\>)|(\?)|(\|)|")
re_with_space = re.compile("(<br\s/><br\s/?)|(-)|(/)|(:).")
def cleantext(var):
    var = re.sub('[0-9]', '', var)
    var = re.sub(r'https?:\/\/\S+', '', var)
    var = re.sub('_', '', var.lower())
    var = re_without_space.sub('', var)
    var = re_with_space.sub(' ', var)
    return var

In [ ]:
data['clean1']=data['text'].apply(str)

In [ ]:
data.head()

In [ ]:
!pip install text2emotion 


In [ ]:
import text2emotion as te

In [ ]:
def happy(var):
    e = te.get_emotion(var)
    return e.get('Happy')

def angry(var):
    e = te.get_emotion(var)
    return e.get('Angry')

def fear(var):
    e = te.get_emotion(var)
    return e.get('Fear')

def sad(var):
    e = te.get_emotion(var)
    return e.get('Sad')

def surprise(var):
    e = te.get_emotion(var)
    return e.get('Surprise')


In [ ]:
data['Happy']=data['clean1'].apply(happy)

In [ ]:
data['Angry']=data['clean1'].apply(angry)
data['Fear']=data['clean1'].apply(fear)
data['Sad']=data['clean1'].apply(sad)
data['Surprise']=data['clean1'].apply(surprise)

In [ ]:
data.head()

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
happy_score=data['Happy'].sum()
angry_score=data['Angry'].sum()
fear_score=data['Fear'].sum()
sad_score=data['Sad'].sum()
surprise_score=data['Surprise'].sum()

In [ ]:
mylabels=['happy','angry','fear','sad','surprise']

In [ ]:
mylabels


In [ ]:
labels = ['happy ['+str(happy_score)+'%]', 'angry ['+str(angry_score)+'%]', 
'fear ['+str(fear_score)+'%]','sad ['+str(sad_score)+'%]','surprise ['+str(surprise_score)+'%]']

total=[happy_score,angry_score,fear_score,sad_score,surprise_score]

colors = ['blue','red','yellow','green','violet']
patches, texts = plt.pie(total, colors=colors, startangle=90)
plt.legend(patches,labels,loc="best")
plt.title("Emotional Pie Chart")
plt.axis('equal')
plt.tight_layout()


